# **DeepFake: Héroe o Villano**

Reemplazar una cara por otra en una foto es una tarea conocida desde hace años para muchos diseñadores gráficos. Pero ahora contamos con una tecnología que permite hacer eso mismo, en tiempo real y en vídeos, con un gran nivel de credibilidad.

A tal punto, que su mal uso se ha denominado DeepFakes aludiendo a las redes neuronales de Deep Learning (Aprendizaje Profundo) que dan vida a esas creaciones falsas (fakes).

Su mal uso ha generado situaciones de acoso, malos entendidos, estrés, depresión y otros males a los que se enfrenta una persona cuando dice que no ha hecho tal cosa, pero hay un vídeo que así lo muestra.

Es, claramente, un villano. ¿O quizás no?

Esta tecnología también puede ser usada tanto para fines comerciales como para fines médicos, mejorando la calidad de vida de sus usuarios y brindando nuevas posibilidades a muchas empresas.


## Objetivo
En este notebook veremos cómo traspasar la cara de un cuerpo a otro, ¡sin cirugía!


## Requisitos
Este notebook requiere una capacidad de procesamiento media, por lo que necesitaremos utilizar una GPU.

Utilizaremos un modelo entrenado que posee un tamaño muy grande, por lo que será necesario conectar Google Drive para acceder a él.


### Google Colaboratory
En el caso de usar Google Colaboratory, es necesario definir un entorno de ejecución con GPU.

Para cambiar nuestro entorno de ejecución a GPU debemos seleccionar en la parte superior del notebook la opción **Entorno de ejecución** y **Cambiar tipo de entorno de ejecución**. En el desplegable **Acelerador por hardware** seleccionamos la opción **GPU** y hacemos clic en **Guardar**.


## Modelo
En esta práctica vamos a utilizar el modelo [First Order Motion Model for Image Animation](https://aliaksandrsiarohin.github.io/first-order-model-website/) propuesto por Aliaksandr Siarohin, Stéphane Lathuilière, Sergey Tulyakov, Elisa Ricci y Nicu Sebe.

Con este modelo se pueden conseguir resultados como el siguiente:

![](https://raw.githubusercontent.com/AliaksandrSiarohin/first-order-model/master/sup-mat/vox-teaser.gif)



## Modelo entrenado

El modelo entrenado tiene un tamaño superior al límite que permite GitHub, por lo que no es posible importarlo de una forma fácil y simple.

La mejor solución hasta el momento pasa por descargar el modelo ya entrenado y subirlo a Google Drive.

El modelo entrenado se denomina `vox-cpk.pth.tar` y lo puedes descargar de alguno de estos enlaces:
- [Modelo entrenado](https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH)
- [Modelo entrenado](https://yadi.sk/d/lEw8uRm140L_eQ)
- [Modelo entrenado](https://www.kaggle.com/xiaoxxiao/damedane-vox-data)

Cuando lo hayas descargardo deberás subirlo a tu Google Drive a una carpeta, por ejemplo `/deepfake-model`. Puedes ponerle otro nombre u otra ruta. Simplemente tenlo en cuenta más adelante, en **Importar modelo entrenado**.


## Acceso a Google Drive
Si el modelo entrenado ya está en su sitio en Google Drive, es necesario conectar este notebook con Google Drive.

Al ejecutar la siguiente celda se abrirá una ventana para conceder los permisos y se nos dará un código de acceso, que debemos copiar y pegar para finalizar la sincronización.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Importar datos de ejemplo

Para poder realizar un deepfake es necesario contar con imágenes y vídeos de entrada para poder trabajar con ellos y poner en el vídeo la imagen de la foto.

En el repositorio de Saturdays.AI Tarragona hay varios ejemplos para que podamos ver su funcionamiento.

In [2]:
!git clone https://github.com/SaturdaysAI/tarragona

Cloning into 'tarragona'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 15560 (delta 8), reused 5 (delta 1), pack-reused 15451
Receiving objects: 100% (15560/15560), 730.04 MiB | 41.79 MiB/s, done.
Resolving deltas: 100% (2825/2825), done.


## Archivos del modelo
Para poder llevar utilizar este modelo es necesario que lo descarguemos, junto a los archivos donde se encuentran las herramientas para utilizarlo.

In [3]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 72.13 MiB | 36.87 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [4]:
cd first-order-model

/content/first-order-model


## Importar librerías
El primer paso en todo script de Python es importar las librerías (en realidad bibliotecas).

Entre ellas encontraremos las típicas numpy y matplotlib, junto a otras quizás menos conocidas.

In [5]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from skimage import img_as_ubyte
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

También será necesario que importemos las librerías propias del modelo.

In [6]:
from demo import load_checkpoints
from demo import make_animation

## Definir archivos de entrada

El modelo necesita una imagen y un vídeo como fuentes para su funcionamiento.

In [33]:
source_image = imageio.imread('/content/tarragona/deepfake-heroe-villano/02.png')
reader = imageio.get_reader('/content/tarragona/deepfake-heroe-villano/04.mp4')

## Redimencionar imagen y vídeo

Necesitamos redimensionar la imagen y el vídeo a un tamaño de 256x256 para que el modelo pueda trabajar de forma adecuada.

In [34]:
# Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []

try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

## Función para unir imagen y vídeo

Utilizaremos esta función para unir la imagen y los vídeos en un solo elemento y así poder ver el funcionamiento de todo el conjunto.

In [35]:
# Mix image and video
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


## Visualizar datos de entrada
A continuación veremos la imagen y el vídeo de entrada, para revisar que podemos continuar con el proyecto.

In [36]:
HTML(display(source_image, driving_video).to_html5_video())

## Importar modelo entrenado
Los datos de un modelo entrenado están disponible en un archivo y en el siguiente paso lo cargaremos en el proyecto.

Es aquí que debes modificar la ruta, si es que lo has puesto en una ruta diferente. En este caso utilizaremos la ruta `/deepfake-model`.

In [11]:
# Load model
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/MyDrive/deepfake-model/vox-cpk.pth.tar')

## **Crear el vídeo**
A llegado la hora de hacer la magia.

Por medio de la función `make_animation` generaremos el vídeo final, basados en el vídeo de entrada (`driving_video`) para que imite sus movimientos, pero con la imagen de entrada (`source_image`) que le hemos dado.

In [37]:
# Create new video
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
imageio.mimsave('/content/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)

100%|██████████| 211/211 [00:07<00:00, 27.95it/s]


## Vídeo final
Para finalizar, vamos a visualizar el vídeo final, junto con la imagen y vídeo de entrada, para poder comparar su funcionamiento en conjunto.

In [38]:
HTML(display(source_image, driving_video, predictions).to_html5_video())

---

## Desafío

¿Te imaginas cómo quedará el vídeo con esta imagen?

![](https://github.com/SaturdaysAI/tarragona/blob/master/deepfake-heroe-villano/cartoon-01.png?raw=true)

**¡Es tu oportunidad de probarlo!**

---